# **Métodos actor-critic e baselines**

**Baselines**

No REINFORCE o gradiente da política é calculado da seguinte forma:


* **Na teoria:** ${\nabla}J \approx \mathbb{E}[Q(s,a){\nabla}log\pi(a|s)]$
* **Na prática (função perda da rede neural):** $\mathcal{L} = -Q(s,a)log\pi(a|s)$

Este gradiente define a direção na qual precisamos mudar os parâmetros da rede para melhorar a política em termos do retorno acumulado total. A escala do próprio gradiente ${\nabla}log\pi(a|s)$ é proporcional ao valor da ação tomada $Q(s,a)$. Isto significa que aumenta-se a probabilidade de escolher ações com maior $Q(s,a)$ e diminui-se a probabilidade de escolher ações com menor $Q(s,a)$. Com relação à fórmula utilizada na prática, como o SGD minimiza a função perda, precisamos colocar o sinal de menos para maximizar o gradiente da política.

O problema desta fórmula para o gradiente da política é que o gradiente tem alta variância. Isto acontece porque o domínio de $Q(s,a)$ depende do ambiente. Considerando um determinado ambiente em que $Q_1$ e $Q_2$ são números positivos e $Q_3$ é um número muito negativo, o gradiente irá maximizar a probabilidade das ações que resultaram $Q_1$ e $Q_2$, e reduzirá muito a probabilidade da ação associada à $Q_3$. 

Porém, considerando agora um ambiente que está associado apenas à recompensas positivas, acrescenta-se uma constante positiva para que os $Q_w$ ($w = 1,2,3$) sejam positivos. Portanto, $Q_1$ e $Q_2$ são números muitos positivos e $Q_3$ é um número positivo muito pequeno quando comparado à $Q_1$ e $Q_2$. Agora o gradiente irá aumentar muito as chances das ações que resultaram em $Q_1$ e $Q_2$ e também irá aumentar um pouco as chances de que a ação associada à $Q_3$ ocorra. 

Neste último ambiente, o incremento da política foi diferente, apesar do fato que as recompensas relativas são as mesmas. Percebe-se que não estamos tentando evitar a ação associada à $Q_3$. Esta dependência entre o incremento da política e a constante adicionada às recompensas pode reduzir a convergência de maneira significativa.  

A abordagem usual para lidar com este problema é subtrair um valor de $Q$. Este valor é denominad **baseline**. Este valor normaliza nosso gradiente da política. Por exemplo considerando duas ações que resultam em $Q_1 = -20$ e $Q_2 = -21$, com o baseline o agente entende que $Q_1$ é relativamente melhor e $Q_2$ é relativamente pior, ao invés de considerar que ambas as escolhas são ruins. Algumas escolhas para o baseline são:

* Uma constante (geralmente a média das recompensas descontadas).
* A média móvel das recompensas descontadas.
* O valor do estado $V(s)$.

**Métodos actor-critic**

A próxima etapa na redução da variância do gradiente da política é tornar a baseline dependente do estado. Pode-se considerar a recompensa total como:

$Q(s,a) = V(s) + A(s,a)$

onde $V(s)$ é o valor do estado e $A(s,a)$ é a vantagem da ação. Utilizando $V(s)$ como baseline, temos:

$\mathcal{L} = -Q(s,a)log\pi(a|s) = -[Q(s,a)-V(s)]log\pi(a|s)$

como $Q(s,a) - V(s) = A(s,a)$, então:

$\mathcal{L} = -A(s,a)log\pi(a|s)$

Portanto a o gradiente será multiplicado pela vantagem da ação, mostrando como a ação $a$ é melhor com relação à média do valor do estado. É uma ótima idéia para melhorar a performance do treinamento, mas não conhecemos $V(s)$. Para resolver isto, podemos utilizar uma segunda rede neural que irá aproximar $V(s)$ para cada observação, minimizando o erro médio quadrático da equação de Bellman de um passo.

**Advantage actor-critic (A2C)**

Utiliza duas redes neurais:

* Actor: é a rede da política, que retorna uma distribuição de probabilidade das ações disponíveis para o agente.

* Critic: Utilizada para saber quão boas são as ações escolhidas.

Abaixo segue um exemplo de como estas redes são combinadas


In [ ]:
class AtariA2C(nn.Module):
'''
  * conv: é uma parte compartilhada pelo ator e pelo critico e está associada ao 
    input de observações (estados)

  * policy: recebe como input o estado s processado por conv e retorna a 
    distribuição de probabilidade das ações para aquele estado s

  * value: recebe como input o estado s processado por conv e retorna uma 
    aproximação de V(s)
'''
  def __init__(self, input_shape, n_actions):
    super(AtariA2C, self).__init__()

    # processamento de observações
    self.conv = nn.Sequential(
      nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=4, stride=2),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1),
      nn.ReLU()
    )
    conv_out_size = self._get_conv_out(input_shape) # pegar o output da parte conv da rede

    # actor
    self.policy = nn.Sequential(
      nn.Linear(conv_out_size, 512),
      nn.ReLU(),
      nn.Linear(512, n_actions)
    )
    # critic
    self.value = nn.Sequential(
      nn.Linear(conv_out_size, 512),
      nn.ReLU(),
      nn.Linear(512, 1)
    )

'''
  Esta função retorna o tamanho do output de conv para construir os inputs
  de policy e value.
'''
def _get_conv_out(self, shape):
  o = self.conv(torch.zeros(1, *shape))
  return int(np.prod(o.size()))

'''
  Percebe-se que forward retorna valores distintos para o ator e 
  para o crítico.
'''
def forward(self, x):
  fx = x.float() / 256
  conv_out = self.conv(fx).view(fx.size()[0], -1)
  return self.policy(conv_out), self.value(conv_out)

$Q(s,a)$ é calculado da seguinte forma:

$Q(s,a) = \big[\sum_{i=0}^{N-1}{\gamma}^ir_i\big] + {\gamma}^NV(S_N)$ 

Este $Q$ será utilizado para calcular a perda $\mathcal{L} = (Q(s,a) - y)^2$ e melhorar a aproximação de $V(s)$ e a vantagem da ação. Para relembrar, no capítulo 6 definiu-se o alvo $y$ como:

$y = r + {\gamma}\max\limits_{a' \in A}Q'(s',a')$

onde $Q'(s,a)$ é a função valor antes do incremento (bootstrap) e $Q(s,a)$ é a função valor incrementada. $N$ é o hiperparâmetro que define quantos passos a frente iremos prever Q(s,a).

In [ ]:
# Para calcular (gamma^i)*r_i
exp_source = ptan.experience.ExperienceSourceFirstLast(
envs, agent, gamma=GAMMA, steps_count=REWARD_STEPS)

In [ ]:
# para calcular gamma^N*V(S_N) (último termo de Q(s,a))
rewards_np = np.array(rewards, dtype=np.float32)
if not_done_idx:
  last_states_v = torch.FloatTensor(np.array(last_states, copy=False)).to(device)
  last_vals_v = net(last_states_v)[1]
  last_vals_np = last_vals_v.data.cpu().numpy()[:, 0]
  last_vals_np *= GAMMA ** REWARD_STEPS
  rewards_np[not_done_idx] += last_vals_np

#  **Exploração em métodos baseados em política**

Diferente dos métodos baseado em valor (ex.: DQN), em métodos baseados em política, não podemos utilizar estratégias como $\epsilon$-greedy para forçar o agente a explorar outras ações aleatoriamente. Isto se deve ao fato que a rede neural (ator) já retorna uma distribuição de probabilidades das ações. Mas ainda sim é possível forçar exploração adicionando um bônus de entropia na função perda.

$
\mathcal{L_H} = {\beta}\sum_i\pi_{\theta}(s_i)log\pi_{\theta}(s_i)
$

O valor desta função é mínimo quando $\pi(s_i)$ é uniforme, portanto, ao adicionarmos entropia para a função perda forçamos o agente a evitar estar tão certo sobre suas ações.

Com todas as funções perda definidas (perda do ator, do crítico e da entropia),
vejamos como fica a atualização dos parâmetros da rede no **A2C**.



In [ ]:
# atualização dos parâmetros no A2C

optimizer.zero_grad()
logits_v, value_v = net(states_v)
loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v) # vals_ref_v = y


# calcular perda do ator para obter o gradiente da política
adv_v = vals_ref_v - value_v.detach() # advantage A(s,a)
log_prob_v = F.log_softmax(logits_v, dim=1) 
log_p_a = log_prob_v[range(BATCH_SIZE), actions_t] # get log(pi(a|s))
log_prob_actions_v = adv_v * log_p_a # A(s,a)*log(pi(a|s))
loss_policy_v = -log_prob_actions_v.mean()  # Exp[-A(s,a)*log(pi(a|s))]

# calcular perda da entropia
prob_v = F.softmax(logits_v, dim=1) # get pi(a|s)
ent = (prob_v * log_prob_v).sum(dim=1).mean() #\sum[pi(a|s)log(pi(a|s))]
entropy_loss_v = ENTROPY_BETA * ent

# Fazer o incremento da política (ator)
loss_policy_v.backward(retain_graph=True)
# extrair gradientes da nossa política para verificar o gradiente máximo, variância e norma L2
grads = np.concatenate([
  p.grad.data.cpu().numpy().flatten()
  for p in net.parameters() if p.grad is not None
])

# Calcular perda do crítico e fazer incremento nos parâmetros da rede do crítco
loss_v = entropy_loss_v + loss_value_v
loss_v.backward()
nn_utils.clip_grad_norm_(net.parameters(),CLIP_GRAD)  
optimizer.step()
loss_v += loss_policy_v

#  **Controle contínuo**

s